In [937]:
import pandas as pd
import numpy as np

In [938]:
df = pd.read_csv('../database/alunos-ativos-da-pos-graduacao.csv', sep=';', usecols=['nome_aluno']).sample(frac=1).reset_index(drop=True)
df = df.rename(columns={'nome_aluno' : 'nome'})

In [939]:
#Atribuindo sexo considerando nomes com maior frequencia
df_nomes_frequencia = df['nome'].map(lambda y: y.split()[0]).value_counts()

nomes_masculinos = ['Lucas', 'João', 'Gabriel', 'Matheus', 'Rafael', 'Guilherme', 'Gustavo', 'Bruno', 'Luiz', 'Paulo', 'Pedro', 'André', 'José', 'Rodrigo', 'Thiago', 'Felipe', 'Daniel', 'Marcelo', 'Eduardo']
nomes_femininos = ['Ana', 'Maria', 'Mariana', 'Bruna', 'Gabriela', 'Camila', 'Fernanda', 'Amanda', 'Aline', 'Leticia', 'Larissa', 'Juliana', 'Jéssica', 'Beatriz', 'Marina']

df['sexo'] = None

df.loc[df['nome'].str.split().str[0].isin(nomes_masculinos), 'sexo'] = 'M'
df.loc[df['nome'].str.split().str[0].isin(nomes_femininos), 'sexo'] = 'F'

lista_sexo_aleatorio = np.random.choice(['M', 'F'], size=df['sexo'].isnull().sum())
df.loc[df['sexo'].isnull(), 'sexo'] = lista_sexo_aleatorio

In [940]:
#Define data de nascimento e idade
df['nascimento'] = None
df.loc[df['nascimento'].isnull(), 'nascimento'] = np.random.choice(pd.date_range(start='1910-01-01', end='2019-12-31').strftime('%Y-%m-%d'), size=len(df))

df['idade'] = (pd.Timestamp.now() - pd.to_datetime(df['nascimento'])).dt.days // 365

In [941]:
#define altura com influencia da idade e sexo
df['altura'] = np.random.randint(120, 210, size=len(df))
df.loc[df['idade'] <= 18, 'altura'] -= (25 * (1 - (df['idade'] / 18) ** 2 )).round()
df.loc[df['idade'] >= 60, 'altura'] -= (30 * ((df['idade'] - 60) / (df['idade'].max() - 60)) ** 2).round()

df.loc[df['sexo'] == 'F', 'altura'] -= np.random.randint(1, 8)
df.loc[df['sexo'] == 'M', 'altura'] += np.random.randint(1, 8)

In [942]:
#define o peso com influencia da idade e sexo
df['peso'] = np.round(np.random.uniform(25.0, 150, size=len(df)), decimals=2)
df.loc[df['idade'] <= 18, 'peso'] -= (8 * (1 - (df['idade'] / 18) ** 2 )).round(decimals=2)
df.loc[df['idade'] >= 60, 'peso'] -= (15 * ((df['idade'] - 60) / (df['idade'].max() - 60)) ** 2).round(decimals=2)

df.loc[df['sexo'] == 'F', 'peso'] -= (df['peso'] * 0.05).round(decimals=2)
df.loc[df['sexo'] == 'M', 'peso'] += (df['peso'] * 0.05).round(decimals=2)

In [943]:
# define pressao arterial
df['pressao-arterial-sistolica'] = 10 * np.random.randint(5, 23, size=len(df))
df['pressao-arterial-diastolica'] = 10 * np.random.randint(3, (df['pressao-arterial-sistolica']/10), size=len(df))

df.loc[df['idade'] <= 18, ['pressao-arterial-sistolica', 'pressao-arterial-diastolica']] -= (10 * np.random.randint(1, 4))
df

,nome,sexo,nascimento,idade,altura,peso,pressao-arterial-sistolica,pressao-arterial-diastolica
0,Dálete Lima de Souza,F,1944-09-29,79,124,114.08,100,70
1,Alexandre Cesar Ricardo,F,1996-10-08,27,136,35.68,190,50
2,Douglas Decicino de Andrade,F,1956-01-22,68,165,107.61,160,50
3,Sara de França Mendes,F,1984-02-22,40,138,131.71,130,50
4,Letícia Munhoz Vellozo Ramos,M,1986-08-21,37,200,75.54,50,30
...,...,...,...,...,...,...,...,...
3880,Luiz Flávio de Queiroz,M,1953-01-10,71,124,50.68,170,120
3881,Homero Alberto Gomes da Silva,M,1992-11-30,31,187,26.96,110,50
3882,Célia Aparecida Almeida Estevam,F,1976-08-27,47,178,93.23,90,80
3883,Braian Garrito Veloso,M,1943-12-08,80,174,117.24,120,100


In [944]:
# define glicemia
df['glicemia'] = 10 * np.random.randint(4, 22, size=len(df))
df

,nome,sexo,nascimento,idade,altura,peso,pressao-arterial-sistolica,pressao-arterial-diastolica,glicemia
0,Dálete Lima de Souza,F,1944-09-29,79,124,114.08,100,70,80
1,Alexandre Cesar Ricardo,F,1996-10-08,27,136,35.68,190,50,190
2,Douglas Decicino de Andrade,F,1956-01-22,68,165,107.61,160,50,150
3,Sara de França Mendes,F,1984-02-22,40,138,131.71,130,50,70
4,Letícia Munhoz Vellozo Ramos,M,1986-08-21,37,200,75.54,50,30,50
...,...,...,...,...,...,...,...,...,...
3880,Luiz Flávio de Queiroz,M,1953-01-10,71,124,50.68,170,120,80
3881,Homero Alberto Gomes da Silva,M,1992-11-30,31,187,26.96,110,50,80
3882,Célia Aparecida Almeida Estevam,F,1976-08-27,47,178,93.23,90,80,180
3883,Braian Garrito Veloso,M,1943-12-08,80,174,117.24,120,100,130


In [945]:
"""

df['frequencia-cardiaca']
df['circunferencia-cintura']
df['diabetes']
df['doenca-cronica']
df['tabagismo']
df['colesterol']
df['funcao-renal']
df['alergia']
df['duracao-sono']"""

"\n\ndf['frequencia-cardiaca']\ndf['circunferencia-cintura']\ndf['diabetes']\ndf['doenca-cronica']\ndf['tabagismo']\ndf['colesterol']\ndf['funcao-renal']\ndf['alergia']\ndf['duracao-sono']"